In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/train.csv', header= 'infer')
Unique_l = labels['individual_id'].unique()

In [ ]:
Unique_l
Unique_l=list(Unique_l)

In [ ]:
print(len(Unique_l))

In [ ]:
if not os.path.exists("/kaggle/temp/"):
    os.mkdir("/kaggle/temp/")

In [ ]:
if not os.path.exists("/kaggle/temp/validation/"):
    os.mkdir("/kaggle/temp/validation/")
if not os.path.exists("/kaggle/temp/train/"):
    os.mkdir("/kaggle/temp/train/")
if not os.path.exists("/kaggle/temp/test/"):
    os.mkdir("/kaggle/temp/test/")

In [ ]:
for i in Unique_l:
    p_t = os.path.join("/kaggle/temp/train/",i)
    if not os.path.exists(p_t):
        os.mkdir(p_t)
        
    p_v = os.path.join("/kaggle/temp/validation/",i)
    if not os.path.exists(p_v):
        os.mkdir(p_v)

In [ ]:
import shutil

In [ ]:
for (int_ind,row) in labels.iterrows():
    img_id = str(row["image"])
    s_p = os.path.join("../input/happywhale-cropped-removebackground-v1/removedBackground_train_images/",img_id)
    p = np.random.random()
    if p <= 0.70:
        t_p = os.path.join("/kaggle/temp/train/",row["individual_id"],img_id)
        shutil.copy(s_p,t_p)
    else:
        t_p = os.path.join("/kaggle/temp/validation/",row["individual_id"],img_id)
        shutil.copy(s_p,t_p)

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Activation,BatchNormalization,GlobalAveragePooling2D,Flatten
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Activation,BatchNormalization,GlobalAveragePooling2D,Flatten
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.applications import EfficientNetB1
base_model = EfficientNetB1(include_top=False,weights='imagenet',pooling='avg')
for layer in base_model.layers:
        layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(15587, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
reduceLROnPlateau = ReduceLROnPlateau(monitor='val_acc',
                                      patience=3,
                                     verbose=1,
                                     factor=0.5,
                                     min_lr=0.00001)

In [ ]:
tr_g = ImageDataGenerator(preprocessing_function=keras.applications.efficientnet.preprocess_input).flow_from_directory(directory='/kaggle/temp/train/',target_size=(240,240) ,classes=Unique_l, batch_size=10, interpolation='bilinear')
v_g = ImageDataGenerator(preprocessing_function=keras.applications.efficientnet.preprocess_input).flow_from_directory(directory='/kaggle/temp/validation/',target_size=(240,240) ,classes=Unique_l, batch_size=10, interpolation='bilinear')

In [ ]:
model.fit(tr_g,epochs=15,callbacks=[reduceLROnPlateau],validation_data=v_g,steps_per_epoch=tr_g.n//tr_g.batch_size,
         validation_steps= v_g.n//v_g.batch_size,workers=8,use_multiprocessing=True)

In [ ]:
model.save('/kaggle/working/model6.h5')

In [ ]:
import os

In [ ]:
if not os.path.exists("/kaggle/temp/test/"):
    os.mkdir("/kaggle/temp/test/")
if not os.path.exists("/kaggle/temp/test/images"):
    os.mkdir("/kaggle/temp/test/images")

In [ ]:
for x in os.listdir('../input/happywhale-cropped-removebackground-v1/removedBackground_test_image/'):
    s_p='../input/happywhale-cropped-removebackground-v1/removedBackground_test_image/'+x
    d_p='/kaggle/temp/test/images/'+x
    if not os.path.exists(d_p):
        shutil.copy(s_p, d_p)

In [ ]:
te_g = ImageDataGenerator(rescale=1/255.,preprocessing_function=keras.applications.efficientnet.preprocess_input).flow_from_directory(
                     directory='/kaggle/temp/test/',
                     target_size=(240,240), 
                     batch_size=10, 
                     shuffle=False, 
                    interpolation='bilinear')

In [ ]:
te_g.reset()
y_predictions = model.predict(te_g)

In [ ]:
predictions_final_class_id=[]
for x in y_predictions:
    temp=[]
    for class_id in range(len(x)):
        temp.append((x[class_id],class_id))
    temp.sort()
    best_5=[temp[-1][1],temp[-2][1],temp[-3][1],temp[-4][1]]
    if temp[-5][0]>=0.05:
        best_5.append(temp[-5][1])
    else:
        best_5.append(-1)
    predictions_final_class_id.append(best_5)
    
predictions_final_class_id=np.array(predictions_final_class_id)

In [ ]:
classes = {value:key for (key,value) in tr_g.class_indices.items()}

predicted_classes=[]

for i in predictions_final_class_id.tolist():
    temp=classes[i[0]]+" "+classes[i[1]]+" "+classes[i[2]]+" "+classes[i[3]]+" "
    temp+="new_individual"
    predicted_classes.append(temp)

In [ ]:
sub = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv',header='infer')
sub['predictions'] = predicted_classes
sub.to_csv('/kaggle/working/submission.csv',index=False)